In [1]:
import time
import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector, Pauli, SparsePauliOp
from qiskit.circuit.library import RealAmplitudes
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
from qiskit.tools.visualization import plot_histogram
# %config InlineBackend.figure_format='retina'

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler, Estimator

In [3]:
service = QiskitRuntimeService(channel='ibm_quantum')

In [4]:
service.backends()

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('simulator_extended_stabilizer')>,
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('ibm_nairobi')>,
 <IBMBackend('ibm_lagos')>,
 <IBMBackend('ibmq_qasm_simulator')>,
 <IBMBackend('simulator_mps')>,
 <IBMBackend('simulator_statevector')>,
 <IBMBackend('ibm_perth')>]

In [28]:
# Make a quantum circuit
qc = QuantumCircuit(4, 3)

# |-> to output
qc.x(3)
qc.h(3)

# |H> to input
qc.h(0)
qc.h(1)
qc.h(2)

# call to oracle
qc.cx(0,3)

# |H> to input
qc.h(0)
qc.h(1)
qc.h(2)

qc.measure(range(3), range(3))

qc.draw()

┌───┐          ┌───┐   ┌─┐
q_0: ┤ H ├───────■──┤ H ├───┤M├
     ├───┤┌───┐  │  └┬─┬┘   └╥┘
q_1: ┤ H ├┤ H ├──┼───┤M├─────╫─
     ├───┤├───┤  │   └╥┘ ┌─┐ ║ 
q_2: ┤ H ├┤ H ├──┼────╫──┤M├─╫─
     ├───┤├───┤┌─┴─┐  ║  └╥┘ ║ 
q_3: ┤ X ├┤ H ├┤ X ├──╫───╫──╫─
     └───┘└───┘└───┘  ║   ║  ║ 
c: 3/═════════════════╩═══╩══╩═
                      1   2  0

In [29]:
# Use local simulator
aer_sim = Aer.get_backend('aer_simulator')
shots = 1024
results = aer_sim.run(qc).result()
answer = results.get_counts()
print(answer)

{'001': 1024}


In [64]:
def bernstein_vazirani(string):
    
    # Save the length of string
    string_length = len(string)
    
    # Make a quantum circuit
    qc = QuantumCircuit(string_length+1, string_length)
    
    # Initialize each input qubit to apply a Hadamard gate and output qubit to |->
    
    # |-> to output
    qc.x(string_length)
    qc.h(string_length)

    # |H> to input
    for i in range(string_length):
        qc.h(i)
    qc.barrier()
    
    # Apply an oracle for the given string
    # Note: In Qiskit, numbers are assigned to the bits in a string from right to left
    for i in range(string_length):
        if(string[i] == '1'):
            qc.cx(i, string_length)
    qc.barrier()
    
    # Apply Hadamard gates after querying the oracle
    
    # |H> to input
    for i in range(string_length):
        qc.h(i)    
    qc.barrier()
    
    # Measurement
    qc.measure(range(string_length), range(string_length))
    
    return qc

In [65]:
qc1 = bernstein_vazirani("111")
qc1.draw()

┌───┐      ░                 ░ ┌───┐ ░ ┌─┐      
q_0: ┤ H ├──────░───■─────────────░─┤ H ├─░─┤M├──────
     ├───┤      ░   │             ░ ├───┤ ░ └╥┘┌─┐   
q_1: ┤ H ├──────░───┼────■────────░─┤ H ├─░──╫─┤M├───
     ├───┤      ░   │    │        ░ ├───┤ ░  ║ └╥┘┌─┐
q_2: ┤ H ├──────░───┼────┼────■───░─┤ H ├─░──╫──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌─┴─┐┌─┴─┐ ░ └───┘ ░  ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─░─┤ X ├┤ X ├┤ X ├─░───────░──╫──╫──╫─
     └───┘└───┘ ░ └───┘└───┘└───┘ ░       ░  ║  ║  ║ 
c: 3/════════════════════════════════════════╩══╩══╩═
                                             0  1  2

In [66]:
qc2 = bernstein_vazirani("000")
qc2.draw()

┌───┐      ░  ░ ┌───┐ ░ ┌─┐      
q_0: ┤ H ├──────░──░─┤ H ├─░─┤M├──────
     ├───┤      ░  ░ ├───┤ ░ └╥┘┌─┐   
q_1: ┤ H ├──────░──░─┤ H ├─░──╫─┤M├───
     ├───┤      ░  ░ ├───┤ ░  ║ └╥┘┌─┐
q_2: ┤ H ├──────░──░─┤ H ├─░──╫──╫─┤M├
     ├───┤┌───┐ ░  ░ └───┘ ░  ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─░──░───────░──╫──╫──╫─
     └───┘└───┘ ░  ░       ░  ║  ║  ║ 
c: 3/═════════════════════════╩══╩══╩═
                              0  1  2

In [67]:
qc3 = bernstein_vazirani("0101")
qc3.draw()

┌───┐      ░            ░ ┌───┐ ░ ┌─┐         
q_0: ┤ H ├──────░────────────░─┤ H ├─░─┤M├─────────
     ├───┤      ░            ░ ├───┤ ░ └╥┘┌─┐      
q_1: ┤ H ├──────░───■────────░─┤ H ├─░──╫─┤M├──────
     ├───┤      ░   │        ░ ├───┤ ░  ║ └╥┘┌─┐   
q_2: ┤ H ├──────░───┼────────░─┤ H ├─░──╫──╫─┤M├───
     ├───┤      ░   │        ░ ├───┤ ░  ║  ║ └╥┘┌─┐
q_3: ┤ H ├──────░───┼────■───░─┤ H ├─░──╫──╫──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌─┴─┐ ░ └───┘ ░  ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────░──╫──╫──╫──╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ░  ║  ║  ║  ║ 
c: 4/═══════════════════════════════════╩══╩══╩══╩═
                                        0  1  2  3

In [68]:
qc4 = bernstein_vazirani("1010")
qc4.draw()

┌───┐      ░            ░ ┌───┐ ░ ┌─┐         
q_0: ┤ H ├──────░───■────────░─┤ H ├─░─┤M├─────────
     ├───┤      ░   │        ░ ├───┤ ░ └╥┘┌─┐      
q_1: ┤ H ├──────░───┼────────░─┤ H ├─░──╫─┤M├──────
     ├───┤      ░   │        ░ ├───┤ ░  ║ └╥┘┌─┐   
q_2: ┤ H ├──────░───┼────■───░─┤ H ├─░──╫──╫─┤M├───
     ├───┤      ░   │    │   ░ ├───┤ ░  ║  ║ └╥┘┌─┐
q_3: ┤ H ├──────░───┼────┼───░─┤ H ├─░──╫──╫──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌─┴─┐ ░ └───┘ ░  ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────░──╫──╫──╫──╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ░  ║  ║  ║  ║ 
c: 4/═══════════════════════════════════╩══╩══╩══╩═
                                        0  1  2  3

In [69]:
backend = service.backends(simulator=True)[0]
print(backend)

<IBMBackend('simulator_extended_stabilizer')>


In [70]:
options = Options(simulator={"seed_simulator": 42}, resilience_level=0) # Do not change values in simulator

In [71]:
with Session(service=service, backend=backend):
    sampler = Sampler(options=options)
    job = sampler.run(circuits=[qc1,qc2,qc3,qc4])

In [75]:
result = job.result()
result

SamplerResult(quasi_dists=[{7: 1.0}, {0: 1.0}, {10: 1.0}, {5: 1.0}], metadata=[{'shots': 4000, 'circuit_metadata': {}}, {'shots': 4000, 'circuit_metadata': {}}, {'shots': 4000, 'circuit_metadata': {}}, {'shots': 4000, 'circuit_metadata': {}}])

In [76]:
respuesta = result.quasi_dists
respuesta

[{7: 1.0}, {0: 1.0}, {10: 1.0}, {5: 1.0}]